In [ ]:
import asyncio
import logging
from datetime import datetime

import influxdb_client
import pandas as pd
from IPython.display import Markdown, display

from tastytrade.config import RedisConfigManager
from tastytrade.config.enumerations import Channels
from tastytrade.connections import Credentials, InfluxCredentials
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.connections.subscription import RedisSubscriptionStore
from tastytrade.messaging.processors import (
    RedisEventProcessor,
    TelegrafHTTPEventProcessor,
)
from tastytrade.utils.time_series import forward_fill

# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

start_time = datetime(2025, 1, 1)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s:%(name)s:%(lineno)d:%(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

# Service Connections


In [ ]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize(force=True)

credentials = Credentials(config=config, env="Live")

dxlink = DXLinkManager(subscription_store=RedisSubscriptionStore())
await dxlink.open(credentials=credentials)

influx_user = InfluxCredentials(config=config)
influxdb = influxdb_client.InfluxDBClient(
    url=influx_user.url, token=influx_user.token, org=influx_user.org
)

# Safely attach processors only if the router is initialized
router = dxlink.router
if router is None:
    raise RuntimeError("DXLink router not initialized after open()")
handlers_dict = getattr(router, "handler", None)
if handlers_dict is None:
    raise RuntimeError("DXLink router.handler mapping not initialized")
for h in handlers_dict.values():
    h.add_processor(TelegrafHTTPEventProcessor())
    h.add_processor(RedisEventProcessor())

# Market Data Subscriptions


In [ ]:
start_time = datetime(2026, 2, 1)

symbols = ["BTC/USD:CXTALP", "NVDA", "AAPL", "QQQ", "SPY", "SPX"]  # , "/ESH25:XCME"]
intervals = ["1d", "1h", "30m", "15m", "5m", "m"]

# uv run tasty-subscription run --symbols "BTC/USD:CXTALP,NVDA,AAPL,QQQ,SPY,SPX" --intervals 1d,1h,30m,15m,5m,m --start-date 2026-01-24 --log-level INFO


# ticker subscriptions
await dxlink.subscribe(symbols)

# candle subscriptions
for symbol in symbols:
    for interval in intervals:
        coroutine = dxlink.subscribe_to_candles(
            symbol=symbol,
            interval=interval,
            from_time=start_time,
        )
        await asyncio.wait_for(coroutine, timeout=60)

In [ ]:
# forward fill
symbols = ["BTC/USD:CXTALP", ".NVDA", "AAPL", "QQQ", "SPY", "SPX"]

for symbol in symbols:
    for interval in intervals:
        event_symbol = f"{symbol}{{={interval}}}"
        logging.debug("Forward-filling %s", event_symbol)
        forward_fill(symbol=event_symbol, lookback_days=5)

# Check | Market Data feeds


In [ ]:
symbol = "SPX{=m}"
# symbol = "BTC/USD:CXTALP{=5m}"
# symbol = "/ESH25:XCME{=m}"

display(Markdown(f"**Candle Feed:** {symbol}"))
router = dxlink.router
if router is None or getattr(router, "handler", None) is None:
    raise RuntimeError("Router or handler map not initialized")
handlers_dict = router.handler
candle_handler = handlers_dict.get(Channels.Candle)
if candle_handler is None:
    raise KeyError("Candle channel handler not available")
feed_proc = candle_handler.processors.get("feed")
if feed_proc is None:
    raise KeyError("'feed' processor missing on Candle handler")
frames = getattr(feed_proc, "frames", {})
if symbol not in frames:
    raise KeyError(f"No frames for symbol {symbol}")
(frames[f"{symbol}"].tail(5).sort(by="time", descending=True))

In [ ]:
display(Markdown(f"**Quote Feed:** {symbol}"))
router = dxlink.router
if router is None or getattr(router, "handler", None) is None:
    raise RuntimeError("Router or handler map not initialized")
quote_handler = router.handler.get(Channels.Quote)
if quote_handler is None:
    raise KeyError("Quote channel handler not available")
feed_proc = quote_handler.processors.get("feed")
if feed_proc is None:
    raise KeyError("'feed' processor missing on Quote handler")
feed_proc.pl

In [ ]:
# TODOS

# [x] Widen the plot
# [x] Remove the scroller at the bottom
# [x] move the legent and remove Price (that is obvious)
# [x] Add MACD
# [x] Fix HULL - Align w/ candlesticks
# [x] ERROR if no study data found

# [ ] Add RSI
# [ ] Add Volume Profile (?? ... /ES, SPY, etc)
# [ ] Add velocity metric

In [ ]:
await dxlink.close()